In [1]:
import os, sys
import numpy as np

bayes_net = [('D', 0.2), ('I', 0.1), ('G|ID', [[0.1, 0.2], [0.2, 0.5]]), ('S|I', [0.3, 0.2]), ('R|G', [0.6, 0.2])]

def make_tensors(formula):
    """ Preprocess probability distributions into form of tensors (axes, 2^len(axes)-array),
          containing the complete distribution with negative cases. """
    return [(axes[0]+axes[2:] if '|' in axes else axes, np.array([np.reshape(1.0 - dist.ravel(), dist.shape), dist])) 
        for axes, dist in ((_, np.array(_dist)) for _, _dist in formula) ]

# Variable Elimination

In [2]:
def ve(order, formula, **query):
    """ Variable Elimination algorithm that computes marginal probabilities for a Bayesian network """
    tensors = make_tensors(formula)
    while order:
        var, order = order[0], order[1:]
        # Leave in the summation for Xi only factors mentioning Xi
        factors = list(filter(lambda i: var in i[0], tensors))
        # Multiply the factors, getting a factor that contains a number for each value of the variables mentioned, including Xi
        axes, tensor_out = ve_product(factors)
        # Sum out Xi, getting a factor f that contains a number for each value of the variables mentioned, not including Xi
        tensor_out = np.sum(tensor_out, axis = axes.index(var))
        axes.remove(var)
        # Replace the multiplied factor in the summation
        tensors = list(filter(lambda i: var not in i[0], tensors))
        tensors.append((''.join(axes), tensor_out))
    if query:
        result_axes, result_tensor = ve_product(tensors)
        return result_tensor[tuple(map(query.get, result_axes))]
    else: # Default query asks for probability of the positive case for the only remaining variable
        return tensors[0][1][1]

def ve_product(factors):
    """ Tensor point-wise multiplication """
    axes = list(set(''.join(map(lambda i: i[0], factors))))
    tensor_out = np.zeros((2,) * len(axes))
    for assignments in range(1<<len(axes)):
        # Maps axes of tensor to a given set of index assignments
        idx = lambda a: tuple(map({v:((assignments>>(len(axes)-1-i))&1) for i, v in enumerate(axes)}.get, a))
        tensor_out[idx(axes)] = np.prod([i[1][idx(i[0])] for i in factors])
    return axes, tensor_out

Run a query by specifying variables and the order to eliminate.

$$P(s) = \sum_{IDGR} P(D)P(I)P(G|ID)P(S|I)P(R|G) = \sum_R \sum_G P(R|G)  \sum_D P(D) \sum_I P(I)P(G|ID)P(S|I)$$

In [26]:
ve('IDGR', bayes_net)

0.29000000000000004

$$P(g) = \sum_{IDSR} P(D)P(I)P(G|ID)P(S|I)P(R|G) = = \sum_R P(R|G) \sum_S P(S|I) \sum_D P(D) \sum_I P(I)P(G|ID)P(S|I) $$

In [4]:
ve('IDSR', bayes_net)

0.13400000000000001

Condictional queries are done by dividing marginal probabilities.

$$P(g|\neg i, \neg d) = \frac{P(g, \neg i, \neg d)}{P(\neg i, \neg d)}=\frac{\sum_{SR}P(\neg d)P(\neg i)P(g|\neg i,\neg d)P(S|\neg i)P(R|g)}{\sum_{GSR}P(\neg d)P(\neg i)P(G|\neg i,\neg d)P(S|\neg i)P(R|G)} $$

In [5]:
ve('SR', bayes_net, G=1, I=0, D=0) / ve('GSR', bayes_net, I=0, D=0)

0.10000000000000001

$$P(i|s) = \frac{P(si)}{P(s)}$$

In [6]:
ve('DGR', bayes_net, S=1, I=1) / ve('IDGR', bayes_net)

0.068965517241379309

$$P(i|r) = \frac{P(ri)}{P(r)}$$

In [7]:
ve('DGS', bayes_net, R=1, I=1) / ve('IDGS', bayes_net)

0.09077598828696927

# Direct Sampling

In [8]:
import random

def ds(order, formula, query, condition = {}):
    """ Direct Sampling algorithm that computes marginal probabilities for a Bayesian network
          Note: `order` must be a topological order of the casual DAG. """
    tensors = make_tensors(formula)
    tensor_index = {axes[0]:(axes, dist) for axes, dist in tensors}
    sample = {v:0 for v in order}
    total_samples = [0]
    def one_sample():
        """ Take one sample with Reject Sampling """
        while 1: # Loop until the sample is not rejected
            for var in order:
                axes, dist = tensor_index[var]
                prob = dist[tuple(map(lambda v:sample[v] if v!=var else 1, axes))]
                sample[var] = int(random.random()<prob)
            total_samples[0] += 1
            if not any(condition[var]!=sample[var] for var in condition):
                break
    counter = 0
    for i in range(100):
        one_sample()
        if all(query[var]==sample[var] for var in query):
            counter += 1
    print("total samples:", total_samples)
    return counter/100

Run a query by specifying the topological order to sample any variables.

In [9]:
ds('IDGSR', bayes_net, dict(S=1))

total samples: [100]


0.34

In [10]:
ds('IDGSR', bayes_net, dict(G=1))

total samples: [100]


0.14

... and the given evidence variables if any.

In [11]:
ds('IDGSR', bayes_net, dict(G=1), dict(I=0, D=0))

total samples: [145]


0.09

In [12]:
ds('IDGSR', bayes_net, dict(I=1), dict(S=1))

total samples: [420]


0.09

In [13]:
ds('IDGSR', bayes_net, dict(I=1), dict(R=1))

total samples: [159]


0.13

# Likelihood Weighting

In [14]:
def ws(order, formula, query, condition = {}):
    """ Likelihood Weighting algorithm that computes marginal probabilities for a Bayesian network
          Note: `order` must be a topological order of unknowns from the casual DAG. """
    tensors = make_tensors(formula)
    tensor_index = {axes[0]:(axes, dist) for axes, dist in tensors}
    sample = {v:0 for v in order}
    def one_sample():
        """ Take one sample with Likelihood Weighting """
        weights = list()
        for var in order:
            axes, dist = tensor_index[var]
            if var not in condition:
                prob = dist[tuple(map(lambda v:sample[v] if v!=var else 1, axes))]
                sample[var] = int(random.random()<prob)
            else:
                sample[var] = condition[var]
                prob = dist[tuple(map(sample.get, axes))]
                weights.append(prob)
        #print(weights)
        return weights
    counter = 0
    for i in range(100):
        W = one_sample()
        if all(query[var]==sample[var] for var in query):
            counter += 1 * np.prod(W)
    return counter/100

In [15]:
ws('IDGSR', bayes_net, dict(S=1))

0.26000000000000001

In [16]:
ws('IDGSR', bayes_net, dict(G=1))

0.17000000000000001

In [17]:
ws('IDGSR', bayes_net, dict(G=1), dict(I=0, D=0))

0.0504

In [18]:
ws('IDGSR', bayes_net, dict(I=1), dict(S=1))

0.017999999999999999

In [19]:
ws('IDGSR', bayes_net, dict(I=1), dict(R=1))

0.050000000000000003

# Markov Chain Monte Carlo

In [20]:
def mcmc(all_vars, formula, query, condition = {}):
    """ Markov Chain Monte Carlo algorithm for bayesian inference. """
    tensors = make_tensors(formula)
    tensor_index = {axes[0]:(axes, dist) for axes, dist in tensors}
    non_evidence = list(set(all_vars)-condition.keys())
    sample = {v:(0 if v in non_evidence else condition[v]) for v in all_vars}
    def one_sample():
        # Pick one non-evidence varible
        var = random.choice(non_evidence)
        axes, dist = tensor_index[var]
        prob = dist[tuple(map(lambda v:sample[v] if v!=var else 1, axes))]
        sample[var] = int(random.random()<prob)
    counter = 0
    for i in range(100):
        one_sample()
        if all(query[var]==sample[var] for var in query):
            counter += 1
    return counter/100

In [21]:
mcmc('IDGSR', bayes_net, dict(S=1))

0.23

In [22]:
mcmc('IDGSR', bayes_net, dict(G=1))

0.19

In [23]:
mcmc('IDGSR', bayes_net, dict(G=1), dict(I=0, D=0))

0.19

In [24]:
mcmc('IDGSR', bayes_net, dict(I=1), dict(S=1))

0.09

In [25]:
mcmc('IDGSR', bayes_net, dict(I=1), dict(R=1))

0.04